#### NN神经网络模型创建 & 模型训练 & 模型验证

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms.v2 import ToTensor     # 转换图像数据为张量
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader  # 数据加载器

In [2]:
# 定义超参数
LR = 1e-3
MAX_EPOCHS = 20
BATCH_SIZE = 128

In [9]:
# 数据集加载
train_dataset = FashionMNIST(root='./fashion_data', train=True, download=True, transform=ToTensor())
test_dataset = FashionMNIST(root='./fashion_data', train=False, download=True, transform=ToTensor())

# 将数据集包装成一个可迭代的对象，方便批量加载数据
trian_data_batch_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)  # shuffle=True表示打乱数据
test_data_batch_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [10]:
# 将多个层按顺序组合在一起，创建一个NN神经网络模型
model = nn.Sequential(
    nn.Linear(784, 64),
    nn.Sigmoid(),
    nn.Linear(64, 10)
)

In [ ]:
# 神经网络模型训练

### 神经网络模型的损失函数
loss_fn = nn.CrossEntropyLoss()  # 交叉熵损失函数

### 创建一个SGD优化器，用于管理模型参数的优化过程
optimizer = torch.optim.SGD(model.parameters(), lr=LR)  # SGD（随机梯度下降）是GD梯度下降算法的一种变体，是PyTorch 深度学习中最常用的优化算法之一。 第一个参数是需要优化的模型参数，通常是通过 model.parameters() 获取

for epoch in range(MAX_EPOCHS):
    #### 批量加载训练数据
    for datas, targets in trian_data_batch_loader:
        ##### 前向运算
        outputs = model(datas.reshape(-1, 784))
        ##### 计算损失
        loss = loss_fn(outputs, targets)
        ##### 反向传播
        optimizer.zero_grad()  # 清空模型参数的梯度（清空模型参数的 .grad 属性）
        loss.backward()     # 计算梯度，计算出的梯度会被保存到模型参数的 .grad 属性中
        optimizer.step()    # 根据计算出的梯度（模型参数的 .grad 属性）更新模型参数

    print(f'Epoch:{epoch} Loss: {loss.item()}')

Epoch:0 Loss: 2.2921106815338135
Epoch:1 Loss: 2.2457001209259033
Epoch:2 Loss: 2.223040819168091
Epoch:3 Loss: 2.189197063446045
Epoch:4 Loss: 2.1689929962158203
Epoch:5 Loss: 2.1303155422210693
Epoch:6 Loss: 2.0892035961151123
Epoch:7 Loss: 2.0608928203582764
Epoch:8 Loss: 2.021317720413208
Epoch:9 Loss: 1.9931696653366089
Epoch:10 Loss: 1.939702033996582
Epoch:11 Loss: 1.91756010055542
Epoch:12 Loss: 1.855538249015808
Epoch:13 Loss: 1.8383941650390625
Epoch:14 Loss: 1.803165316581726
Epoch:15 Loss: 1.7507432699203491
Epoch:16 Loss: 1.6898185014724731
Epoch:17 Loss: 1.6940175294876099
Epoch:18 Loss: 1.612195372581482
Epoch:19 Loss: 1.5848363637924194


In [27]:
# 模型验证
correct = 0
total = 0
with torch.no_grad():  # 不计算梯度
    for datas, real_labels in test_data_batch_loader:
        outputs = model(datas.reshape(-1, 784))
        _, predict_labels= torch.max(outputs, 1)  # 返回 1（列）维度的最大值及其索引
        total += real_labels.size(0)  # size(0) 等效 shape[0]
        correct += (predict_labels == real_labels).sum().item()   # item()作用是将单元素张量中的值转换为标准的 Python 标量

print(f'Accuracy: {correct/total*100}%')

Accuracy: 62.96000000000001%


In [28]:
# 数据观察
print(predict_labels.shape)
print(predict_labels)
print(real_labels)
print(outputs.shape)
print(outputs)

torch.Size([16])
tensor([3, 1, 7, 5, 7, 2, 9, 2, 8, 9, 1, 9, 1, 3, 1, 7])
tensor([3, 2, 7, 5, 8, 4, 5, 6, 8, 9, 1, 9, 1, 8, 1, 5])
torch.Size([16, 10])
tensor([[ 6.1565e-01,  8.4695e-01, -1.8687e-01,  1.0305e+00,  1.6089e-01,
         -5.5101e-01,  1.4671e-01, -8.6142e-01, -5.2219e-01, -1.0906e+00],
        [ 2.3099e-01,  3.0338e-01,  1.3966e-01,  2.6335e-01,  2.3699e-01,
         -3.0825e-01,  2.1673e-01, -4.2621e-01, -2.0000e-01, -4.9406e-01],
        [-1.1875e+00, -6.5277e-01, -4.0597e-01, -8.5361e-01, -5.4064e-01,
          1.3349e+00, -4.7986e-01,  1.6743e+00,  7.6120e-01,  9.7126e-01],
        [-6.3345e-01, -4.7233e-01, -2.3001e-01, -5.6015e-01, -3.5503e-01,
          8.8611e-01, -2.5039e-01,  6.1920e-01,  4.2348e-01,  8.3075e-01],
        [-8.4474e-01, -8.6412e-01, -1.9029e-02, -9.7825e-01, -1.9212e-01,
          9.5022e-01, -2.0681e-01,  1.1285e+00,  1.0014e+00,  9.1366e-01],
        [ 3.9304e-01, -8.1164e-02,  6.5074e-01,  8.3177e-02,  6.3914e-01,
         -5.4854e-01,  6.2236